# 멀티 프로세스와 멀티 스레드

## 멀티 프로세스

- 멀티 프로세스는 별도의 메모리 영역을 가진다. 
- 각 프로세스는 독립적인 stack, heap, code, data영역을 가진다. 
- 특별한 메커니즘으로만 통신 가능하다. (IPC: inter process communication), 시그널, 메시지 큐, 파이프, 파일 
- 파이썬에서는 subprocess 모듈을 사용한다. 

## 멀티 스레드 

- 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근한다. 
- 스레드는 데이터 공유를 통해 간단하게 통신한다. 
- threading 모듈 처리를 통해 한번에 한 스레드만 메모리 영역에 접근할 수 있다. 
- 한 프로세스에 속한 스레드는 stack영역을 제외한 메모리 영역을 공유한다.
- GIL?

## 동시성(concurrency)

- 논리적으로 여러 작업이 동시에 실행되는 것처럼 보이는 것이다. 
- 예를 들어 I/O(파일 및 네트워크 소켓 입력 및 출력)연산 등은 프로그램의 흐름에 큰 짐이 될 수 있다. 
- 이럴 때 한 작업의 I/O연산이 완료되기를 기다리는 동안 다른 작업을 수행하여 유휴 시간을 활용하는 것이 동시성이다. 

## 병렬성(parallelism)

- 물리적으로 여러 작업이 동시에 처리되는 것이다. 
- 데이터 병렬성과 작업 병렬성으로 나눌 수 있다. 
- 데이터 병렬성은 같은 작업을 병렬처리하는 것이다. 
- 하나의 커다란 작업에서 전체 데이터를 쪼갠 후 병렬처리하면 작업을 빠르게 수행할 수 있다. 
- 작업 병렬성은 서로 다른 작업을 병렬처리하는 것이다. 웹 서버에서는 다수의 독립적인 요청을 병렬로 개별적으로 처리할 수 있다. 

## 6.1.1 subprocess 모듈

subprocess모듈은 부모-자식 프로세스 쌍을 생성하는 데 사용된다. 
부모 프로세스는 사용자에 의해 실행된다. 
부모 프로세스는 차례로 다른 일을 처리하는 자식 프로세스의 인스턴스를 실행한다. 
자식 프로세스를 사용함으로써, 멀티코어의 이점을 최대한 취하고 동시성 문제를 운영체제가 알아서 처리하도록 한다. 

In [1]:
import subprocess
subprocess.run(['echo', '이것은 subprocess입니다.'])

CompletedProcess(args=['echo', '이것은 subprocess입니다.'], returncode=0)

In [3]:
subprocess.CompletedProcess(args=['echo', '이것은 subprocess입니다.'], returncode=0)

CompletedProcess(args=['echo', '이것은 subprocess입니다.'], returncode=0)

In [4]:
subprocess.run(['sleep','10'])

CompletedProcess(args=['sleep', '10'], returncode=0)

## 6.1.2 threading 모듈

스레드가 여러 개로 분리되면 스레드 간 데이터 공유의 복잡성이 증가한다. 또한 락과 데드락을 회피하는데 주의를 기울여야한다. 
파이썬 프로그램에는 단 하나의 메인 스레드만 존재한다. 멀티스레드를 사용하려면 threading모듈을 사용한다. 

내부적으로 락을 관리하려면 queue모듈을 사용한다. 큐에 의존하면 자원의 접근을 직렬화할 수 있고, 이는 곧 한 번에 하나의 스레드만 데이터에 접근할 수 있게 한다는 뜻이다. 실행 중인 스레드가 있는 동안에는 프로그램은 종료되지 않는다. 

워커 스레드가 작업을 완료했는데도 프로그램이 종료되지 않고 계속 실행되는 경우 문제가 될 수 있다. 스레드를 데몬으로 변환하면 데몬 스레드가 실해되지 않는 즉시 프로그램이 종료된다. 

queue.join()메서드는 큐가 빌 때 까지 (큐의 모든 항목이 처리될 때까지)기다린다.

In [5]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        print("스레드 {0}: 처리 완료 {1}".format(num+1, item))
        q.task_done()

if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i, ))
        t.start()
        threads.append(t)
    
    for item in range(20):
        q.put(item)
    
    q.join()

    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

스레드 3: 처리 완료 0스레드 1: 처리 완료 1스레드 5: 처리 완료 2스레드 2: 처리 완료 3스레드 4: 처리 완료 4
스레드 1: 처리 완료 5
스레드 1: 처리 완료 6
스레드 1: 처리 완료 7
스레드 1: 처리 완료 8
스레드 1: 처리 완료 9
스레드 1: 처리 완료 10
스레드 1: 처리 완료 11
스레드 1: 처리 완료 12
스레드 1: 처리 완료 13
스레드 1: 처리 완료 14
스레드 1: 처리 완료 15
스레드 1: 처리 완료 16
스레드 1: 처리 완료 17
스레드 1: 처리 완료 18
스레드 1: 처리 완료 19






## Queue (FIFO)

In [ ]:
import queue

q = queue.Queue()
ql = []

for i in range(5):
    q.put(i)
    ql.append(i)

while not q.empty():
    print(q.get())

print(ql)

## Queue(LIFO) = Stack

In [ ]:
import queue

q = queue.LifoQueue()
ql = []

for i in range(5):
    q.put(i)
    ql.append(i)

while not q.empty():
    print(q.get())
print(ql)

## Priority Queue

In [ ]:
import threading, queue

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()

# turn-on the worker thread
threading.Thread(target=worker, daemon=True).start()

# send thirty task requests to the worker
for item in range(30):
    q.put(item)
print('All task requests sent\n', end='')

# block until all tasks are done
q.join()
print('All work completed')

## 제곱과 세제곱 구하는 함수.

In [ ]:
import time

num = 11
numbers = range(1, num)

def square(numbers):
    for n in numbers:
        time.sleep(1)
        if n == num-1:
            print('square', n, n*n)

def cube(numbers):
    for n in numbers:
        time.sleep(1)
        if n == num-1:
            print('cube',n, n*n*n)

start = time.time()
square(numbers)
cube(numbers)
end = time.time()
time = end - start
time

In [ ]:
import time
import threading

num = 11
numbers = range(1, num)

def square(numbers):
    for n in numbers:
        time.sleep(1)
        if n == num-1:
            print('square', n, n*n)

def cube(numbers):
    for n in numbers:
        time.sleep(1)
        if n == num-1:
            print('cube',n, n*n*n)


start = time.time()
t1 = threading.Thread(target=square, args=(numbers,))
t2 = threading.Thread(target=cube, args=(numbers,))
t1.start()
t2.start()
t1.join()
t2.join()

cube(numbers)
end = time.time()
time = end - start
time

## Thread

In [ ]:
import time

def countdown(n):
    while n > 0:
        print('Count Down', n)
        n -= 1
        time.sleep(3)

from threading import Thread
t = Thread(target=countdown, args=(10, ))
t.start()

## process

In [2]:
import os

print(os.getpid()) # process ID
print(os.getuid()) # user ID
print(os.getgid()) # group ID
print(os.getcwd()) # current directory

28698
501
20
/Users/jooyoungson/Sample/PythonProgramming/basic
